In [52]:
import openai
from serpapi import GoogleSearch
from dotenv import load_dotenv
import os
import json
from flask import jsonify

In [ ]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
SERP_API_KEY = os.getenv('SERP_API_KEY')

In [ ]:
def google_shopping_search(query, num_results = 5):
    params = {
        "api_key": SERP_API_KEY,
        "engine": "google_shopping",
        "q": query,
        "hl": "en",
        "gl": "sg",
        "num": num_results
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    products = []

    for item in results.get("shopping_results", []):
        product = {
            "name": item.get("title"),
            "product_url": item.get("link"),
            "shop_name": item.get("source"),
            "image_url": item.get("thumbnail"),
            "price": item.get("price")
        }
        products.append(product)

    return products


In [ ]:
results = google_shopping_search("shirts <$100", 5)
for product in results:
    print(product)


{'name': "100% Cotton Men's Casual Thick T-Shirt Japanese Style Festival Loose Oversized ...", 'product_url': 'https://shopee.sg/product/329084101/15800323859?gads_t_sig=VTJGc2RHVmtYMTlxTFVSVVRrdENkY0N5akVpcE5OamJEdjRRTHBOTzhmbmtuN0M4eG4vYVZ2ejkxVkYrbmZ2Sm8zYzdXcGpZc0YwRUZQbDU4a1V6VFh4KytGNG51N2ZOTS9zRjF0cWRibThYL1doYTVBUDBIRzVCVHhHN215OWc', 'shop_name': 'Shopee', 'image_url': 'https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcTQddDCez3LWkXifNxj7Mhsqkmk4Fjp5YxPuBSlbRhiufIL_5vDPSEmHI-O9cFKcr2H2rpvcVFcWd36uDK1eaYCNvIfIg6WvtCYPHOuQa9R&usqp=CAE', 'price': '$17.43'}
{'name': 'DLSTAR 100% Cotton Polo Short Sleeve T-Shirt - Casual Homewear Comfy Tee ...', 'product_url': 'https://www.lazada.sg/products/dlstar-100-cotton-polo-short-sleeve-t-shirt-casual-homewear-comfy-tee-activewear-sports-professional-officewear-i2505839638-s20739942619.html?from_gmc=1&fl_tag=1', 'shop_name': 'Lazada Singapore', 'image_url': 'https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcSbgVTfb6M_nSAA32-OwWKWg

In [25]:
def create_keyword(category, preferences):
    prompt = f"category: {category}. preferences: {preferences}"
    system_msg = """You are a helpful assistant for fashion and shopping recommendations. Based on the clothes category and preferences, generate json format of an effective keyword to be used in serpapi google shopping search api and a short sentence of response.
                    JSON format:
                    {
                        "keyword": the keyword,
                        "response": 1 short sentence of response (example: Here are the top 5 recommended shirt products)
                    }"""

    response = openai.ChatCompletion.create(
        model = "gpt-4o",

        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response['choices'][0]['message']['content'].strip()

In [26]:
print(create_keyword("shirts", "I love green color, and i'm poor."))

```json
{
    "keyword": "affordable green shirts",
    "response": "Here are budget-friendly green shirt options for you."
}
```


In [59]:
def generate_response(category, preferences = ""):
    prompt = f"category: {category}. preferences: {preferences}"
    system_msg = """You are a helpful assistant for fashion and shopping recommendations. Based on the clothes category and preferences, generate json format of an effective keyword to be used in serpapi google shopping search api and a short sentence of response. Starts the response with '{' and ends with '}'
                    The format: {
                        "keyword": <the keyword>
                        "response": <1 short sentence of response> (example: Here are the recommended blue shirts:)
                    }
                """

    response = openai.ChatCompletion.create(
        model = "gpt-4o",

        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ]
    )

    return response['choices'][0]['message']['content'].strip()


In [66]:
gpt_response = generate_response("shirt", "I love green color and I am poor.")
gpt_response = json.loads(gpt_response)
print(gpt_response)
keyword = gpt_response['keyword']
response = gpt_response['response']
print(f"keyword: {keyword}")
print(f"response: {response}")

{'keyword': 'affordable green shirts', 'response': 'Here are some affordable green shirts for you:'}
keyword: affordable green shirts
response: Here are some affordable green shirts for you:


In [28]:
def rephrase_details(search_results):
    system_msg = """You are a helpful assistant that is good at shortening description. User will input search results in json format, you must also return the same json format but with shortened text.
                    JSON format:
                    {
                        "name": (make sure it is 5 to 8 inclusive words),
                        "product_url": (don't need to shorten this, just return the same url),
                        "shop_name": (make sure it is 1 or 2 words and not url),
                        "image_url": (don't need to shorten this, just return the same url),
                        "price": (don't need to shorten this, just return the same price)
                    }"""

    response = openai.ChatCompletion.create(
        model = "gpt-4o",

        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": search_results}
        ]
    )
    
    return response['choices'][0]['message']['content'].strip()

In [33]:
search_results = json.dumps(google_shopping_search("shirts"))

print(search_results)
print(rephrase_details(search_results))

[{"name": "Buy Men's Floral Printed Formal Shirt Online @ womensfashionfun M / Charcoal ...", "product_url": "https://www.womensfashionfun.com/products/mens-floral-printed-formal-shirt-sf_885charcoal?variant=45878129000618&country=SG&currency=SGD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&srsltid=AfmBOooGLN-WS2KGLbervv-bVTZaTdfQ5SlgxMaWyJe-jzuFV5-1iTkmmKE", "shop_name": "Womens Fashion Fun", "image_url": "https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcQ_KUcotvVyAhzuJjcxG5Qi1RpllpyYHtuEeIzqlFGjFDcWZafObluoMrep7kcx52LeEs2-dWmDjjjKhLRPO1tINRZbQBv3_lM5oERvp6Ne24Sdc_wDi6sC&usqp=CAE", "price": "$34.00"}, {"name": "Buy Men's Floral Printed Formal Shirt Online @ womensfashionfun L / Grey / Polycotton", "product_url": "https://www.womensfashionfun.com/products/mens-floral-printed-formal-shirt-sf_885grey?variant=45878127296682&country=SG&currency=SGD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic

In [ ]:
def shorten_text(text, min, max):
    system_msg = "You are a helpful assistant that is good at shortening text."
    prompt = f"Shorten this text into {min} to {max} (inclusive) words: {text}"

    response = openai.ChatCompletion.create(
        model = "gpt-4o",

        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response['choices'][0]['message']['content'].strip()